# Sentiment Analysis of Twitter Data

### installing required libraries
twint library used for colliction data from twitter without API.

In [1]:
!pip install twint
!pip install --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
#this prevents async problems/ runtime errors
!pip install nest_asyncio

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.3/385.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.3 MB/s eta 0:00:00
  Created wheel for twint: filename=twint-2.1.20-py3-none-any.whl size=33929 sha256=18fd96d9da04be852152bae2ba08546b52d7b5b668467cd8fc92b012a3dc64a9
  Stored in directory: /root/.cache/pip/wheels/44/fc/77/99887a36b5c265a87516158858697d1a0b8f32c4d4dbddbb24
  Created wheel for googletransx: filename=googletransx-2.4.2-py3-none-any.whl size=15968 sha256=03d61b3017f9c93016e0844d5893d512d0cb769f4963c49b5236f16405f6c2fa
  Stored 

### import required libraries

In [2]:
import twint
import nest_asyncio
nest_asyncio.apply()
import json
import re

### Collection data
Instantiate and configure the twint-object for get data from twitter.


In [3]:
c = twint.Config()
c.Store_object = True
c.Pandas =True
c.Search = "#سياحة"
# c.Since = '2022-01-01'
# c.Until = '2022-11-30'
c.Limit = 100
c.Lang = 'ar'
c.Filter_retweets = False
c.Store_json = True
c.Output = "1_tweets.json"
twint.run.Search(c)
# Quick check
# twint.storage.panda.Tweets_df.head(20)

1602425573820596224 2022-12-12 22:10:03 +0000 <pro_travel1> ندعوك للاستمتاع باجواء الصيف معانا فى جورجيا واتساب: 00995596355000 #سياحة #سفر  https://t.co/RChE1Qkmyf
1602423571984228352 2022-12-12 22:02:05 +0000 <QalWQalt> افضل #سياحة عائلية في اذربيجان واتساب: 00994508225050  https://t.co/EwXSgILFpE
1602412284629946375 2022-12-12 21:17:14 +0000 <safari_news> بالصور .. استمتع بجولة سياحية في مدينة #بوردو الفرنسية   https://t.co/iP6mmFZg3W  #سياحة  https://t.co/CBifB82g9B
1602411766205530113 2022-12-12 21:15:11 +0000 <Nkhatiib> السلام عليكم ،  شركة سياحية تحتاج موظفين سعودين لهم خبرة في المجال ، الي مهتم ياليت يتواصل معاي على الخاص وشكرًا . #وظائف #سياحة #سفر #تنظيم #منظمين.
1602409197274124288 2022-12-12 21:04:58 +0000 <safari_news> #زامبيا تعفي #السعوديين من تأشيرة الدخول لأراضيها   https://t.co/AfD1g10AUq  #سياحة  https://t.co/IP6AHulZys
1602406697347256322 2022-12-12 20:55:02 +0000 <safari_news> روكسي سينما تفتتح شاشتها السينمائية على سطح غاليريا مول البرشاء   https://t.co/IL3vJC9S63

## 1- Loading and Cleaning Twitter Data

In [4]:
# Open the input file
input_file = open('/kaggle/input/sentiment-arabic-tweets/1_tweets.json','r', encoding="utf-8")
clean_tweets = open('/kaggle/working/clean_tweets.txt','w', encoding="utf-8")


# Load the first few lines
for line in input_file:
    tweet = json.loads(line)
    temp = re.sub("@[A-Za-z0-9_]+","@User", tweet['tweet'])
    temp = re.sub(r"http\S+", "URL", temp)
    temp = re.sub(r"www.\S+", "URL", temp)
    temp = re.sub('[()!\'?]', '', temp)
    temp = re.sub('\[.*?\]','', temp)
    temp = re.sub("[^أ-يA-z0-9@]"," ", temp)
    clean_tweets.write(temp + '\n')

# Close the input file
input_file.close()
clean_tweets.close()